In [4]:
import ee
import geemap

In [5]:
Map_S5P = geemap.Map()
Map_S5P

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [6]:
point_S5P = ee.Geometry.Rectangle([-122.8003, 37.4831, -122.8036, 37.8288])

image_S5P = ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_NO2") \
    .filterBounds(point_S5P) \
    .filterDate('2018-07-11', '2019-06-30') \
    .sort('ALGORITHM_VERSION') \
    .first() \
    .select('NO2_column_number_density')

vis_params = {
    'min': 0,
    'max': 0.0002,
    'bands': ['NO2_column_number_density'],
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}
Map_S5P.centerObject(point_S5P, 8)
Map_S5P.addLayer(image_S5P, vis_params, "Sentinel-5P TROPIMI NO2")


In [7]:
Map_LANDSAT8 = geemap.Map()
Map_LANDSAT8

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [8]:
point_LANDSAT8 = ee.Geometry.Point([-122.4439, 37.7538])

image_LANDSAT8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(point_LANDSAT8) \
    .filterDate('2018-07-01', '2019-06-30') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map_LANDSAT8.centerObject(point_LANDSAT8, 8)
Map_LANDSAT8.addLayer(image_LANDSAT8, vis_params, "Landsat-8")


In [9]:
ee.Date(image_LANDSAT8.get('system:time_start')).format('YYYY-MM-dd').getInfo()

'2018-11-08'

In [11]:
#image.get('ALGORITHM_VERSION').getInfo()

In [12]:
#tropimi = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(image_S5P.geometry())
Map_LANDSAT8.addLayer(image_S5P, {}, 'TROPIMI')
Map_LANDSAT8

Map(bottom=25636.0, center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['positio…

In [13]:
# Make the training dataset.
points = image_S5P.sample(**{
    'region': image_LANDSAT8.geometry(),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map_LANDSAT8.addLayer(points, {}, 'training', False)

In [14]:
print(points.size().getInfo())

5000


In [15]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-121.68730640159825, 37.86722226931812]}, 'id': '0', 'properties': {'NO2_column_number_density': 0.00011661837197607383}}


In [19]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'NO2_column_number_density'

# Overlay the points on the imagery to get training.
training = image_LANDSAT8.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().setOutputMode('REGRESSION').train(training, label, bands)

In [20]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 652, 'B2': 889, 'B3': 1427, 'B4': 2152, 'B5': 3359, 'B6': 3883, 'B7': 2410, 'NO2_column_number_density': 0.00011661837197607383}}


In [21]:
# Classify the image with the same bands used for training.
result = image_LANDSAT8.select(bands).classify(trained)

# # Display the clusters with random colors.
Map_LANDSAT8.addLayer(result.randomVisualizer(), {}, 'classfied')
Map_LANDSAT8

Map(bottom=25636.0, center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['positio…